# Freqtrade Programmatic Backtesting Analysis

This notebook demonstrates how to run backtesting and analyze results programmatically without using the CLI.

## 1. Setup and Imports

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Freqtrade imports
from freqtrade.configuration import Configuration
from freqtrade.data.history import load_pair_history
from freqtrade.resolvers import StrategyResolver
from freqtrade.data.dataprovider import DataProvider
from freqtrade.data.btanalysis import (
    load_backtest_stats,
    load_trades_from_db,
    analyze_trade_parallelism,
    extract_trades_of_period
)

print("Imports successful!")

## 2. Load Configuration

In [ ]:
# Define paths
user_data_dir = Path("../").resolve()
config_path = user_data_dir / "config.json"

print(f"User data directory: {user_data_dir}")
print(f"Config path: {config_path}")
print(f"Config exists: {config_path.exists()}")

In [ ]:
# Load configuration from file
config = Configuration.from_files([str(config_path)])

# Override some settings for backtesting
config['strategy'] = 'HyperoptStrategy'  # Change to your strategy
config['timeframe'] = '1h'
config['datadir'] = user_data_dir / 'data' / 'binance'

print(f"Loaded config for exchange: {config.get('exchange', {}).get('name', 'unknown')}")
print(f"Strategy: {config['strategy']}")
print(f"Timeframe: {config['timeframe']}")
print(f"Data directory: {config['datadir']}")

## 3. Load Strategy

In [ ]:
# Load the strategy
strategy = StrategyResolver.load_strategy(config)

# Initialize DataProvider (required for some strategies)
strategy.dp = DataProvider(config, None, None)

print(f"Strategy loaded: {strategy.name}")
print(f"Timeframe: {strategy.timeframe}")
print(f"Stoploss: {strategy.stoploss}")
print(f"Trailing stop: {strategy.trailing_stop}")
print(f"Minimal ROI: {strategy.minimal_roi}")

## 4. Load Historical Data

In [ ]:
# Select a pair to analyze
pair = "BTC/USDT"

# Load candle data
candles = load_pair_history(
    datadir=config['datadir'],
    timeframe=config['timeframe'],
    pair=pair,
    data_format='feather'  # or 'json' depending on your data format
)

print(f"Loaded {len(candles)} candles for {pair}")
print(f"Date range: {candles['date'].min()} to {candles['date'].max()}")
candles.tail()

## 5. Analyze Strategy Signals

In [ ]:
# Run strategy analysis on the candles
df = strategy.analyze_ticker(candles, {"pair": pair})

print(f"Analysis complete. DataFrame shape: {df.shape}")
print(f"\nColumns added by strategy:")
strategy_columns = [col for col in df.columns if col not in candles.columns]
print(strategy_columns)

In [ ]:
# Show entry signals
entry_signals = df[df['enter_long'] == 1]
print(f"Total entry signals: {len(entry_signals)}")
entry_signals[['date', 'open', 'high', 'low', 'close', 'volume', 'rsi', 'macd', 'enter_long']].head(10)

In [ ]:
# Show exit signals
exit_signals = df[df['exit_long'] == 1]
print(f"Total exit signals: {len(exit_signals)}")
exit_signals[['date', 'open', 'high', 'low', 'close', 'volume', 'rsi', 'exit_long']].head(10)

## 6. Visualize Indicators and Signals

In [ ]:
# Plot price with entry/exit signals
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Limit data for visualization
plot_df = df.tail(500).copy()
plot_df = plot_df.set_index('date')

# Price chart with Bollinger Bands
ax1 = axes[0]
ax1.plot(plot_df.index, plot_df['close'], label='Close', linewidth=1)
if 'bb_lower' in plot_df.columns:
    ax1.fill_between(plot_df.index, plot_df['bb_lower'], plot_df['bb_upper'], alpha=0.2, label='BB')

# Mark entry signals
entries = plot_df[plot_df['enter_long'] == 1]
ax1.scatter(entries.index, entries['close'], marker='^', color='green', s=100, label='Entry', zorder=5)

# Mark exit signals
exits = plot_df[plot_df['exit_long'] == 1]
ax1.scatter(exits.index, exits['close'], marker='v', color='red', s=100, label='Exit', zorder=5)

ax1.set_title(f'{pair} - Price with Entry/Exit Signals')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# RSI
ax2 = axes[1]
if 'rsi' in plot_df.columns:
    ax2.plot(plot_df.index, plot_df['rsi'], label='RSI', color='purple')
    ax2.axhline(y=70, color='r', linestyle='--', alpha=0.5)
    ax2.axhline(y=30, color='g', linestyle='--', alpha=0.5)
    ax2.fill_between(plot_df.index, 30, 70, alpha=0.1)
ax2.set_title('RSI')
ax2.set_ylim(0, 100)
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.3)

# MACD
ax3 = axes[2]
if 'macd' in plot_df.columns and 'macd_signal' in plot_df.columns:
    ax3.plot(plot_df.index, plot_df['macd'], label='MACD', color='blue')
    ax3.plot(plot_df.index, plot_df['macd_signal'], label='Signal', color='orange')
    ax3.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
ax3.set_title('MACD')
ax3.legend(loc='upper left')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Run Full Backtesting Programmatically

In [ ]:
from freqtrade.optimize.backtesting import Backtesting

# Prepare config for backtesting
backtest_config = Configuration.from_files([str(config_path)])

# Set backtesting parameters
backtest_config['strategy'] = 'HyperoptStrategy'
backtest_config['timeframe'] = '1h'
backtest_config['timerange'] = '20251001-'  # From October 2025
backtest_config['datadir'] = user_data_dir / 'data' / 'binance'
backtest_config['exportfilename'] = user_data_dir / 'backtest_results' / 'notebook_backtest.json'
backtest_config['export'] = 'trades'
backtest_config['stake_amount'] = 100  # USDT per trade
backtest_config['dry_run_wallet'] = 1000  # Starting balance

# Limit pairs for faster testing (optional)
# backtest_config['pairs'] = ['BTC/USDT', 'ETH/USDT']

print("Backtest configuration ready")
print(f"Strategy: {backtest_config['strategy']}")
print(f"Timerange: {backtest_config.get('timerange', 'all')}")

In [ ]:
# Run backtesting
# Note: This may take a while depending on the data size
backtesting = Backtesting(backtest_config)
backtesting.start()

## 8. Load and Analyze Backtest Results

In [ ]:
# Load backtest results from file
backtest_results_dir = user_data_dir / 'backtest_results'

# List available backtest files
backtest_files = list(backtest_results_dir.glob('*.json'))
print("Available backtest result files:")
for f in backtest_files:
    print(f"  - {f.name}")

In [ ]:
# Load the most recent backtest result
if backtest_files:
    latest_backtest = max(backtest_files, key=lambda x: x.stat().st_mtime)
    print(f"Loading: {latest_backtest.name}")
    
    stats = load_backtest_stats(str(latest_backtest))
    
    # Get strategy stats
    strategy_stats = stats.get('strategy', {})
    strategy_name = list(strategy_stats.keys())[0] if strategy_stats else None
    
    if strategy_name:
        results = strategy_stats[strategy_name]
        print(f"\nStrategy: {strategy_name}")
        print(f"Total trades: {results.get('total_trades', 0)}")
        print(f"Profit total: {results.get('profit_total', 0):.4f}")
        print(f"Profit total abs: {results.get('profit_total_abs', 0):.2f} USDT")
        print(f"Win rate: {results.get('wins', 0) / max(results.get('total_trades', 1), 1) * 100:.1f}%")
        print(f"Max drawdown: {results.get('max_drawdown', 0) * 100:.2f}%")
else:
    print("No backtest results found. Run backtesting first.")

In [ ]:
# Load trades as DataFrame
if backtest_files and strategy_name:
    trades_df = pd.DataFrame(results.get('trades', []))
    
    if not trades_df.empty:
        print(f"Total trades: {len(trades_df)}")
        print(f"\nTrade columns: {list(trades_df.columns)}")
        
        # Show trade summary
        trades_df[['pair', 'open_date', 'close_date', 'profit_abs', 'profit_ratio', 'exit_reason']].head(10)

## 9. Performance Visualization

In [ ]:
if 'trades_df' in dir() and not trades_df.empty:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Cumulative profit
    ax1 = axes[0, 0]
    trades_df['cumulative_profit'] = trades_df['profit_abs'].cumsum()
    ax1.plot(trades_df['cumulative_profit'], linewidth=2)
    ax1.set_title('Cumulative Profit (USDT)')
    ax1.set_xlabel('Trade #')
    ax1.set_ylabel('Profit (USDT)')
    ax1.grid(True, alpha=0.3)
    ax1.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    
    # 2. Profit distribution
    ax2 = axes[0, 1]
    trades_df['profit_abs'].hist(bins=30, ax=ax2, edgecolor='black')
    ax2.set_title('Profit Distribution per Trade')
    ax2.set_xlabel('Profit (USDT)')
    ax2.set_ylabel('Count')
    ax2.axvline(x=0, color='r', linestyle='--', alpha=0.5)
    
    # 3. Profit by pair
    ax3 = axes[1, 0]
    pair_profits = trades_df.groupby('pair')['profit_abs'].sum().sort_values()
    colors = ['red' if x < 0 else 'green' for x in pair_profits]
    pair_profits.plot(kind='barh', ax=ax3, color=colors)
    ax3.set_title('Total Profit by Pair')
    ax3.set_xlabel('Profit (USDT)')
    ax3.axvline(x=0, color='gray', linestyle='-', alpha=0.5)
    
    # 4. Exit reasons
    ax4 = axes[1, 1]
    exit_counts = trades_df['exit_reason'].value_counts()
    exit_counts.plot(kind='pie', ax=ax4, autopct='%1.1f%%')
    ax4.set_title('Exit Reasons')
    ax4.set_ylabel('')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Trade duration analysis
if 'trades_df' in dir() and not trades_df.empty:
    trades_df['open_date'] = pd.to_datetime(trades_df['open_date'])
    trades_df['close_date'] = pd.to_datetime(trades_df['close_date'])
    trades_df['duration_hours'] = (trades_df['close_date'] - trades_df['open_date']).dt.total_seconds() / 3600
    
    print(f"Average trade duration: {trades_df['duration_hours'].mean():.1f} hours")
    print(f"Min duration: {trades_df['duration_hours'].min():.1f} hours")
    print(f"Max duration: {trades_df['duration_hours'].max():.1f} hours")
    print(f"Median duration: {trades_df['duration_hours'].median():.1f} hours")

## 10. Compare Multiple Strategies

In [ ]:
def compare_strategies(result_files):
    """Compare multiple backtest results."""
    comparison = []
    
    for file_path in result_files:
        try:
            stats = load_backtest_stats(str(file_path))
            strategy_stats = stats.get('strategy', {})
            
            for strategy_name, results in strategy_stats.items():
                total_trades = results.get('total_trades', 0)
                wins = results.get('wins', 0)
                
                comparison.append({
                    'Strategy': strategy_name,
                    'File': file_path.name,
                    'Total Trades': total_trades,
                    'Win Rate': f"{wins / max(total_trades, 1) * 100:.1f}%",
                    'Profit (USDT)': f"{results.get('profit_total_abs', 0):.2f}",
                    'Profit %': f"{results.get('profit_total', 0) * 100:.2f}%",
                    'Max Drawdown': f"{results.get('max_drawdown', 0) * 100:.2f}%",
                    'Sharpe': f"{results.get('sharpe', 0):.2f}",
                    'Sortino': f"{results.get('sortino', 0):.2f}"
                })
        except Exception as e:
            print(f"Error loading {file_path}: {e}")
    
    return pd.DataFrame(comparison)

# Compare all available backtest results
if backtest_files:
    comparison_df = compare_strategies(backtest_files)
    display(comparison_df)

## 11. Indicator Statistics

In [ ]:
# Analyze indicator values at entry points
if 'df' in dir() and 'enter_long' in df.columns:
    entry_df = df[df['enter_long'] == 1].copy()
    
    print("Indicator statistics at ENTRY signals:")
    print("=" * 50)
    
    indicators = ['rsi', 'macd', 'macd_signal']
    for ind in indicators:
        if ind in entry_df.columns:
            print(f"\n{ind.upper()}:")
            print(f"  Mean: {entry_df[ind].mean():.2f}")
            print(f"  Std:  {entry_df[ind].std():.2f}")
            print(f"  Min:  {entry_df[ind].min():.2f}")
            print(f"  Max:  {entry_df[ind].max():.2f}")

In [ ]:
# Analyze indicator values at exit points
if 'df' in dir() and 'exit_long' in df.columns:
    exit_df = df[df['exit_long'] == 1].copy()
    
    print("Indicator statistics at EXIT signals:")
    print("=" * 50)
    
    indicators = ['rsi', 'macd', 'macd_signal']
    for ind in indicators:
        if ind in exit_df.columns:
            print(f"\n{ind.upper()}:")
            print(f"  Mean: {exit_df[ind].mean():.2f}")
            print(f"  Std:  {exit_df[ind].std():.2f}")
            print(f"  Min:  {exit_df[ind].min():.2f}")
            print(f"  Max:  {exit_df[ind].max():.2f}")

## 12. Custom Analysis Functions

In [ ]:
def analyze_strategy_performance(trades_df):
    """Generate comprehensive performance metrics."""
    if trades_df.empty:
        return {"error": "No trades to analyze"}
    
    total_trades = len(trades_df)
    winning_trades = len(trades_df[trades_df['profit_abs'] > 0])
    losing_trades = len(trades_df[trades_df['profit_abs'] < 0])
    
    metrics = {
        "Total Trades": total_trades,
        "Winning Trades": winning_trades,
        "Losing Trades": losing_trades,
        "Win Rate": f"{winning_trades / total_trades * 100:.1f}%",
        "Total Profit": f"{trades_df['profit_abs'].sum():.2f} USDT",
        "Average Profit": f"{trades_df['profit_abs'].mean():.2f} USDT",
        "Best Trade": f"{trades_df['profit_abs'].max():.2f} USDT",
        "Worst Trade": f"{trades_df['profit_abs'].min():.2f} USDT",
        "Avg Win": f"{trades_df[trades_df['profit_abs'] > 0]['profit_abs'].mean():.2f} USDT" if winning_trades > 0 else "N/A",
        "Avg Loss": f"{trades_df[trades_df['profit_abs'] < 0]['profit_abs'].mean():.2f} USDT" if losing_trades > 0 else "N/A",
    }
    
    # Calculate profit factor
    gross_profit = trades_df[trades_df['profit_abs'] > 0]['profit_abs'].sum()
    gross_loss = abs(trades_df[trades_df['profit_abs'] < 0]['profit_abs'].sum())
    metrics["Profit Factor"] = f"{gross_profit / max(gross_loss, 0.01):.2f}"
    
    return metrics

if 'trades_df' in dir() and not trades_df.empty:
    metrics = analyze_strategy_performance(trades_df)
    print("\nPerformance Metrics:")
    print("=" * 40)
    for key, value in metrics.items():
        print(f"{key:20}: {value}")

---
## Notes

- Make sure you have downloaded historical data before running backtesting
- Adjust `timerange` parameter to limit the backtesting period
- Use `config['pairs']` to limit pairs for faster testing
- Results are saved to `backtest_results/` directory